# Vessel Route Optimization

In [1]:
import math
import numpy as np
from numpy import random
from scipy.optimize import fsolve
import ortools
from tabulate import tabulate
from ortools.linear_solver import pywraplp

## Multiple array problem

In [3]:
# THIS FUNCTION DEFINES THE GRID WE WILL BE USING THROUGHOUT THE WHOLE EXERCISE, AS WELL AS IT GENERATES THE
# DIFFERENT RANDOM VALUES FOR THE WIND

def random_space(max_value, x_positions=5, y_positions=1, day_max_change=0.3, position_max_change=0.2, seed=None):    
    random.seed(seed)
    degrees_matrix = []
    wind_degrees = []
    
    for day in range(x_positions):
        wind_degrees.append(random.randint(1,max_value))
    degrees_matrix.append(wind_degrees) 

    for position in range(0,y_positions-1):
        position_degree = []
        for day in range(x_positions):
            same_day_other_position = abs(degrees_matrix[position-1][day])  
            if same_day_other_position != 0:
                position_degree.append(random.randint(same_day_other_position*(1-position_max_change),same_day_other_position*(1+position_max_change)))
            else: 
                position_degree.append(random.randint(1,max_value))
        degrees_matrix.append(position_degree)
    
    return degrees_matrix

x_positions = 5
y_positions = 3
day_max_change = 0.3
position_max_change = 0.3
seed = 126

wind_direction = random_space(360, x_positions, y_positions, day_max_change, position_max_change)
wind_speed = random_space(27, x_positions, y_positions, day_max_change, position_max_change)

print("Wind direction (degrees):", wind_direction)
print("Wind speed (m/s):", wind_speed)

Wind direction (degrees): [[210, 307, 151, 70, 266], [189, 395, 158, 59, 317], [235, 223, 169, 84, 332]]
Wind speed (m/s): [[10, 15, 5, 22, 17], [10, 16, 4, 16, 19], [10, 16, 5, 15, 16]]


# Open_sail function 

In [4]:
# THIS FUNCTION CHECKS THE OPENNING OF THE SAILS DEPENDING ON THE WIND ANGLE AND STRENGTH GIVEN IN INPUT 

# Constants adapted to the context of the problem
min_wind_angle = 60 
max_wind_angle = 300 
min_wind_strength = 10

def open_sail(wind_angle, wind_strength):
    open_sail_bool = False 
    
    if wind_angle > min_wind_angle and wind_angle < max_wind_angle and wind_strength > min_wind_strength :
        open_sail_bool = True
        
    # EXCLUDING CRIETRIA FOR THE BOAT_SPEED FUNCTION NOT TO EXPLODE
    if wind_angle == 180 : open_sail_bool = False
    if wind_angle > 90 and wind_angle < 110 : open_sail_bool = False
    if wind_angle > 270 and wind_angle < 290 : open_sail_bool = False


    return open_sail_bool


print("We open the sail : ", open_sail(80,40))

We open the sail :  True


# Apparent wind function for boat speed

In [5]:
# THIS FUNCTION COMPUTES THE SPEED GENERATED BY THE SAILS DEPENDING ON THE ANGLE OF THE WIND, ITS STRENGTH
# AND THE BOAT PARAMETER ETA

def boat_speed(a0,VW,eta): 

    f = lambda a: math.sin(math.pi*a0/180) * math.sin(math.pi*a/180) * math.pow((math.sin(math.pi*a/180/2) / math.sin(math.pi*a0/180-math.pi*a/180)), 2)-VW * eta
    a = fsolve(f, [0])[0]
    #print("a:", a)
    VB = VW * (math.sin(math.pi*(a0-a)/180)/math.sin(math.pi*a/180))
    #print("a0:",a0,"and VB:", VB)
    
    return VB

# Problem in 1 dimension basic



In [23]:
#THIS PROBLEM ...

#External input
total_distance = 2925 #miles

#Choice of granularity of the grid 
x_positions = 6 #nb of squares between A and B

#Constants
eta = 0.01
motor_speed = 25.3 #in mph
motor_emission_per_mile = 750 #in kgCO2 per mile (100% motor)
cost_per_mile = 65 #dollars per mile (100% motor)

wind_angle = random_space(360,1,x_positions) #in degrees
wind_speed = random_space(30,1,x_positions) #in mph
square_length = total_distance/x_positions

#defining the lists to be used
list_sail = []
list_boat_speed = []
list_co2 = []
list_time = []

#flatten the list of list into a simple list
wind_angle = [item for sublist in wind_angle for item in sublist]
wind_speed = [item for sublist in wind_speed for item in sublist]

#evaluating if we open the sails for each position
for i in range(x_positions):
    list_sail.append(open_sail(wind_angle[i],wind_speed[i]))
    
#determining the boat speeds depending on whether we use sails on motor   
for i in range(x_positions):
    if list_sail[i]==True:
        list_boat_speed.append(boat_speed(wind_angle[i],wind_speed[i],eta))
        
    else:
        list_boat_speed.append(motor_speed)
        
#CO2 emissions calculations
for i in range(x_positions):
    if list_sail[i]==True:
        list_co2.append(0)
        
    else:
        list_co2.append(motor_emission_per_mile*square_length)
        
total_emissions = sum(list_co2)
        
#determining the time needed for the journey
for i in range(x_positions):
    list_time.append(square_length/list_boat_speed[i])
        
total_time = sum(list_time)        
        
    
    
#printing results
print("The squares length is", square_length)
print("Wind angles are :", wind_angle)
print("Wind speeds are :", wind_speed)
print("We open the sails :", list_sail)
print("Boat speeds are :", list_boat_speed)
print("The emissions for each square are:", list_co2)
print("The total emissions are:", total_emissions)
print("The time need for each square is:", list_time)
print("The total time needed is:", total_time)

The squares length is 487.5
Wind angles are : [113, 113, 111, 134, 109, 132]
Wind speeds are : [13, 12, 12, 12, 10, 9]
We open the sails : [True, True, True, True, False, False]
Boat speeds are : [16.165546633323927, 15.326185732011524, 15.32067797026166, 14.738808570217834, 25.3, 25.3]
The emissions for each square are: [0, 0, 0, 0, 365625.0, 365625.0]
The total emissions are: 731250.0
The time need for each square is: [30.15672844585777, 31.808305636135394, 31.819740676376483, 33.07594353217079, 19.268774703557312, 19.268774703557312]
The total time needed is: 165.39826769765506


# Problem in 1 dimension improved

In [33]:
#External input
total_distance = 2925 #miles

#Choice of granularity of the grid 
x_positions = 6 #nb of squares between A and B

#Constants
eta = 0.5
motor_speed = 25.3 #in mph
motor_emission_per_mile = 750 #in kgCO2 per mile (100% motor)
cost_per_mile = 65 #dollars per mile (100% motor)

wind_angle = random_space(360,1,x_positions) #in degrees
wind_speed = random_space(30,1,x_positions) #in mph
square_length = total_distance/x_positions

#defining the lists to be used
list_co2 = []
list_time = []
list_sail_speed = []
list_motor_percentage = []
list_boat_speed = []

#flatten the list of list into a simple list
wind_angle = [item for sublist in wind_angle for item in sublist]
wind_speed = [item for sublist in wind_speed for item in sublist]

#evaluating the increase of speed we can reach thanks to sails 
for i in range(x_positions):
    if open_sail(wind_angle[i],wind_speed[i]):
        speed = boat_speed(wind_angle[i],wind_speed[i],eta)
        if 10 > speed > 0.1*motor_speed :
            list_sail_speed.append(speed)
        else: 
            list_sail_speed.append(0)
    else: list_sail_speed.append(0)

#calculating the power needed of the motor in % to reach 20mph
list_motor_percentage = [((motor_speed-i)/motor_speed) for i in list_sail_speed]

#calculating the sum of the speeds (sail + motor)
list_boat_speed = [list_sail_speed[i] + (list_motor_percentage[i]*motor_speed) for i in range(len(list_sail_speed))]
 
#CO2 emissions calculations
for i in range(x_positions):
    list_co2.append(motor_emission_per_mile*list_motor_percentage[i]*square_length)
    
total_emissions = sum(list_co2)
        
#determining the time needed for the journey
for i in range(x_positions):
    list_time.append(square_length/list_boat_speed[i])
        
total_time = sum(list_time)  

total_co2_no_sail = motor_emission_per_mile * total_distance

real_cost = 0
for i in range(x_positions) : real_cost += cost_per_mile*square_length*list_motor_percentage[i]

total_cost_no_sail = cost_per_mile * total_distance
    
#printing results
print("The squares length is", square_length)
print("Wind angles are :", wind_angle)
print("Wind speeds are :", wind_speed)

print("Speed gained from sails:", list_sail_speed)
print("Motor usage:",  list_motor_percentage)
print("Boat speeds are :", list_boat_speed)

print("The emissions for each square are:", list_co2)
print("The total emissions are:", total_emissions)
print("The time need for each square is:", list_time)
print("The total time needed is:", total_time)

print("\nCO2:")
print("The total emitted CO2 is",total_emissions,"kgCO2")
print("The spared CO2 is",total_co2_no_sail - total_emissions,"kgCO2")

print("\nCOSTS:")
print("The total cost of the trip is", real_cost,"dollars")
print("The spared cost compared to a full-motor trip is", total_cost_no_sail - real_cost,"dollars")


The squares length is 487.5
Wind angles are : [108, 121, 119, 128, 117, 134]
Wind speeds are : [12, 12, 10, 10, 10, 10]
Speed gained from sails: [0, 3.626592651466252, 0, 0, 0, 0]
Motor usage: [1.0, 0.8566564169380929, 1.0, 1.0, 1.0, 1.0]
Boat speeds are : [25.3, 25.300000000000004, 25.3, 25.3, 25.3, 25.3]
The emissions for each square are: [365625.0, 313215.00244299026, 365625.0, 365625.0, 365625.0, 365625.0]
The total emissions are: 2141340.0024429904
The time need for each square is: [19.268774703557312, 19.26877470355731, 19.268774703557312, 19.268774703557312, 19.268774703557312, 19.268774703557312]
The total time needed is: 115.61264822134387

CO2:
The total emitted CO2 is 2141340.0024429904 kgCO2
The spared CO2 is 52409.99755700957 kgCO2

COSTS:
The total cost of the trip is 185582.80021172581 dollars
The spared cost compared to a full-motor trip is 4542.199788274185 dollars


# Problem in 2 dimension

In [15]:
#External input
total_distance = 2925 #miles

#Choice of granularity of the grid 
x_positions = 17 #nb of squares between A and B
y_positions = 10

#Constants
eta = 0.05
motor_speed = 25.3 #in mph
motor_emission_per_mile = 750 #in kgCO2 per mile (100% motor)
cost_per_mile = 65 #dollars per mile (100% motor)



square_length = total_distance/x_positions

wind_angle_matrix = random_space(360, x_positions, y_positions, seed=126) #in degrees
wind_speed_matrix = random_space(30, x_positions, y_positions, seed=149) #in mph


def single_line(wind_angle, wind_speed, print_=False):
    
    #defining the lists to be used
    list_co2 = []
    list_time = []
    list_sail_speed = []
    list_motor_percentage = []
    list_boat_speed = []

    #evaluating the increase of speed we can reach thanks to sails 
    for i in range(x_positions):
        if open_sail(wind_angle[i],wind_speed[i]):
            speed = boat_speed(wind_angle[i],wind_speed[i],eta)
            if 10 > speed > 0.1*motor_speed :
                list_sail_speed.append(speed)
            else: 
                list_sail_speed.append(0)
        else: list_sail_speed.append(0)

    #calculating the power needed of the motor in % to reach 20mph
    list_motor_percentage = [((motor_speed-i)/motor_speed) for i in list_sail_speed]

    #calculating the sum of the speeds (sail + motor)
    list_boat_speed = [list_sail_speed[i] + (list_motor_percentage[i]*motor_speed) for i in range(len(list_sail_speed))]

    #CO2 emissions calculations
    for i in range(x_positions):
        list_co2.append(motor_emission_per_mile*list_motor_percentage[i]*square_length)

    total_emissions = sum(list_co2)

    #determining the time needed for the journey
    for i in range(x_positions):
        list_time.append(square_length/list_boat_speed[i])

    total_time = sum(list_time)
    
    #printing results
    if print_:
        print("The squares length is", square_length)
        print("Wind angles are :", wind_angle)
        print("Wind speeds are :", wind_speed)

        print("Speed gained from sails:", list_sail_speed)
        print("Motor usage:",  list_motor_percentage)
        print("Boat speeds are :", list_boat_speed)

        print("The emissions for each square are:", list_co2)
        print("The total emissions are:", total_emissions)
        print("The time need for each square is:", list_time)
        print("The total time needed is:", total_time)
        
    return (wind_angle, wind_speed, list_sail_speed, list_boat_speed, list_co2, list_motor_percentage)

path = np.zeros((y_positions, x_positions), int)
result_wind_angle = []
result_wind_speed = []
result_sail_speed = []
result_boat_speed = []
result_co2 = []
result_motor_percentage = []

for y in range(y_positions):
    results = single_line(wind_angle_matrix[y], wind_speed_matrix[y])
    result_wind_angle.append(results[0])
    result_wind_speed.append(results[1])
    result_sail_speed.append(results[2])
    result_boat_speed.append(results[3])
    result_co2.append(results[4])
    result_motor_percentage.append(results[5])
    
result_co2

## New test

solver_kind = pywraplp.Solver.BOP_INTEGER_PROGRAMMING
solver = pywraplp.Solver("default",solver_kind)
#solver = pywraplp.Solver.CreateSolver('CLP')

x = {}
for i in range(y_positions):
    for j in range(x_positions):
        #x[i, j] = solver.Var(0, 1, 'x')
        #x[i, j] = solver.IntVar(0, 1, 'x')
        x[i, j] = solver.BoolVar('x')

# Define the constraints
middle_point = middle_point = y_positions // 2 
# The starting and ending points are fixed
solver.Add(x[middle_point,0] == 1)
solver.Add(x[middle_point,x_positions-1] == 1)


r = [middle_point-1, middle_point, middle_point+1]



# For each column, only one variable can take the value 1
for j in range(x_positions):
    solver.Add(solver.Sum([x[i,j] for i in range(y_positions)]) == 1)
    # Force second column constraint to follow from middle point and be have only allowed potential solutions
    


# For each column, the only allowed movements from the previous column are
# to the same row, the up-diagonal, or the low-diagonal
for j in range(1, x_positions):
    for i in range(y_positions):
        
        if i > 0 and i < y_positions-1:
            solver.Add(x[i,j] <= x[i,j-1] + x[i-1,j-1] + x[i+1,j-1])
        if i == 0:
            solver.Add(x[i,j] <= x[i,j-1] + x[i+1,j-1])
        if i == y_positions-1:
            solver.Add(x[i,j] <= x[i,j-1] + x[i-1,j-1])
        if j == 1:
            solver.Add(solver.Sum([x[i, j] for i in r]) == 1)


#for j in range(1, x_positions):
 #   for i in range(y_positions):
  #      solver.Add(x[i,j] == 0 or x[i,j] == 1)            
            
# for each x_position, we need the boat to pass by there
for j in range(x_positions):
    solver.Add(solver.Sum([x[i, j] for i in range(y_positions)]) == 1)





solver.Minimize(solver.Sum([result_co2[i][j] * x[i,j] for i in range(y_positions)
                                                  for j in range(x_positions)]))
status = solver.Solve()



# Print solution.
if status == pywraplp.Solver.OPTIMAL:
    total_co2=solver.Objective().Value()
    print('Total co2 = ', solver.Objective().Value(), '\n')
    for j in range(x_positions):
        for i in range(y_positions):
            # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
            if x[i, j].solution_value() >= 0.5:
                print('x_position %d took y_position %d.  CO2 = %d' %
                      (j, i, result_co2[i][j]))
                

for j in range(x_positions):
        for i in range(y_positions):
            # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
            path[i,j] = (x[i,j].solution_value())

def show_table(path):
    results = path.tolist()          
    for i in range(len(results)):
        for j in range(len(results[i])):
            if results[i][j] == 1:
                results[i][j] = "#"
            elif results[i][j] == 0:
                results[i][j] = ""
    print(tabulate(list(map(tuple, results)), tablefmt="grid", stralign="center"))

show_table(path)




dist_traveled = 0
# Calculating the total distance
for j in range(1, x_positions):
    for i in range(y_positions):
        if path[i,j] == 1:
            if i > 0 and i < y_positions-1:
                if path[i,j-1] == 1:
                    dist_traveled += square_length
                if path[i-1,j-1]==1 or path[i+1,j-1]==1:
                    dist_traveled += math.sqrt(2)*square_length
                    
            if i == 0:
                if path[i,j-1] == 1:
                    dist_traveled += square_length
                if path[i+1,j-1]==1:
                    dist_traveled += math.sqrt(2)*square_length
                    
            if i == y_positions-1:
                if path[i,j-1] == 1:
                    dist_traveled += square_length
                if path[i-1,j-1]==1:
                    dist_traveled += math.sqrt(2)*square_length
                    
total_time = dist_traveled / motor_speed
time_diff = total_time - total_distance/motor_speed 
time_change = time_diff / (total_distance/motor_speed)*100

def decimal_to_hours(decimal_hours):
    hours = int(decimal_hours)
    minutes = int((decimal_hours - hours) * 60)
    return f"{hours}h {minutes:02d}min"


change_distance = (dist_traveled-total_distance )/total_distance*100
gained_co2 = motor_emission_per_mile*total_distance-total_co2
change_emissions = (total_co2-motor_emission_per_mile*total_distance)/total_co2*100

line_cost = cost_per_mile*total_distance
real_cost = line_cost*(1+change_emissions*0.01)

                    
print("\nDISTANCE:")
print("The distance traveled by the boat is",round(dist_traveled,2),"miles, and the total distance between A and B was", total_distance, "miles")                
print("Increase of distance: ", round(change_distance,2),"%")

print("\nTIME:")
print("The total time spent is",decimal_to_hours(total_time))
print("The total time lost is",decimal_to_hours(time_diff))
print("Increase in time:",round(time_change,2),"%")

print("\nCO2:")
print("The total CO2 emitted is",round(total_co2,2),"kgCO2")
print("The gained CO2 compared to boat with no boat is", round(gained_co2,2) , "kgCO2")
print("Decrease in CO2:", round(change_emissions,2),"%")

print("\nCOSTS:")
print("The fuel cost of the trip is", real_cost, "dollars")
print("The spared cost is", line_cost-real_cost, "dollars")

                        

Total co2 =  1920049.0614385935 

x_position 0 took y_position 5.  CO2 = 94788
x_position 1 took y_position 5.  CO2 = 80205
x_position 2 took y_position 5.  CO2 = 79561
x_position 3 took y_position 4.  CO2 = 129044
x_position 4 took y_position 3.  CO2 = 129044
x_position 5 took y_position 4.  CO2 = 129044
x_position 6 took y_position 5.  CO2 = 78276
x_position 7 took y_position 6.  CO2 = 129044
x_position 8 took y_position 7.  CO2 = 129044
x_position 9 took y_position 8.  CO2 = 129044
x_position 10 took y_position 9.  CO2 = 81595
x_position 11 took y_position 8.  CO2 = 129044
x_position 12 took y_position 9.  CO2 = 86135
x_position 13 took y_position 8.  CO2 = 129044
x_position 14 took y_position 7.  CO2 = 129044
x_position 15 took y_position 6.  CO2 = 129044
x_position 16 took y_position 5.  CO2 = 129044
+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+---+--

In [9]:
result_sail_speed
                
                

[[0, 0, 0, 8.479773317773471, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0,
  0,
  0,
  8.144134662279319,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7.7532670059438535,
  0,
  0,
  0,
  0],
 [0, 0, 0, 0, 0, 9.761980816631745, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7.127564489747253],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [6.715973800410478,
  9.575159473532745,
  9.70145655731597,
  0,
  0,
  0,
  9.95329448876114,
  0,
  0,
  0,
  0,
  0,
  7.704703343704261,
  0,
  0,
  0,
  0],
 [8.946229748435531, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0,
  8.744511508860054,
  7.922700579743033,
  0,
  0,
  8.613205902895588,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  6.023112717437917],
 [0,
  0,
  0,
  0,
  0,
  9.21840560217191,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  6.930741859129379],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  9.30264927320574,
  0,
  8.412445859884267,
  0,
  0,
  0,
  8.4

# FINAL PROBLEM

In [16]:
show_table(path)

+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|   |   |   |   | # |   |   |   |   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|   |   |   | # |   | # |   |   |   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
| # | # | # |   |   |   | # |   |   |   |   |   |   |   |   |   | # |
+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   | # |   |   |   |   |   |   |   | # |   |
+---+---+---+---+---

In [11]:
print(wind_speed)

[28, 25, 24, 24, 26, 27]
